# Machine translation

Name: Paulo Victor Correia
Student ID: 2167525

The goal of this TP is to build a machine translation model.
You will be comparing the performance of three different architectures:
* A vanilla RNN
* A GRU-RNN
* A transformer

You are provided with the code to load and build the pytorch dataset,
and the code for the training loop.
You "only" have to code the architectures.
Of course, the use of built-in torch layers such as `nn.GRU`, `nn.RNN` or `nn.Transformer`
is forbidden, as there would be no exercise otherwise.

The source sentences are in english and the target language is french.

This is also for you the occasion to see what a basic machine learning pipeline looks like.
Take a look at the given code, you might learn a lot!

Do not forget to **select the runtime type as GPU!**

**Sources**

* Dataset: [Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/)

<!---
M. Cettolo, C. Girardi, and M. Federico. 2012. WIT3: Web Inventory of Transcribed and Translated Talks. In Proc. of EAMT, pp. 261-268, Trento, Italy. pdf, bib. [paper](https://aclanthology.org/2012.eamt-1.60.pdf). [website](https://wit3.fbk.eu/2016-01).
-->

* The code is inspired by this [pytorch tutorial](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html).

*This notebook is quite big, use the table of contents to easily navigate through it.*

# Imports and data initializations

We first download and parse the dataset. From the parsed sentences
we can build the vocabularies and the torch datasets.
The end goal of this section is to have an iterator
that can yield the pairs of translated datasets, and
where each sentences is made of a sequence of tokens.

## Imports

In [ ]:
!python3 -m spacy download en > /dev/null
!python3 -m spacy download fr > /dev/null
!pip install torchinfo > /dev/null
!pip install einops > /dev/null
!pip install wandb > /dev/null


from itertools import takewhile
from collections import Counter, defaultdict

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, Vocab
from torchtext.datasets import IWSLT2016

import einops
import wandb
from torchinfo import summary
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

2023-03-08 20:46:49.342764: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 20:46:49.497965: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-08 20:46:50.251946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-08 20:46:50.252054: W tensorflow/compiler/xla/stream_executor

The tokenizers are objects that are able to divide a python string into a list of tokens (words, punctuations, special tokens...) as a list of strings.

The special tokens are used for a particular reasons:
* *\<unk\>*: Replace an unknown word in the vocabulary by this default token
* *\<pad\>*: Virtual token used to as padding token so a batch of sentences can have a unique length
* *\<bos\>*: Token indicating the beggining of a sentence in the target sequence
* *\<eos\>*: Token indicating the end of a sentence in the target sequence

In [ ]:
# Original dataset, but there's a bug on Colab with it
# train, valid, _ = IWSLT2016(language_pair=('fr', 'en'))
# train, valid = list(train), list(valid)

# Another dataset, but it is too huge
# !wget https://www.statmt.org/wmt14/training-monolingual-europarl-v7/europarl-v7.en.gz
# !wget https://www.statmt.org/wmt14/training-monolingual-europarl-v7/europarl-v7.fr.gz
# !gunzip europarl-v7.en.gz
# !gunzip europarl-v7.fr.gz

# with open('europarl-v7.en', 'r') as my_file:
#     english = my_file.readlines()

# with open('europarl-v7.fr', 'r') as my_file:
#     french = my_file.readlines()

# dataset = [
#     (en, fr)
#     for en, fr in zip(english, french)
# ]
# print(f'\n{len(dataset):,} sentences.')

# dataset, _ = train_test_split(dataset, test_size=0.8, random_state=0)  # Remove 80% of the dataset (it would be huge otherwise)
# train, valid = train_test_split(dataset, test_size=0.2, random_state=0)  # Split between train and validation dataset

# Our current dataset
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip


df = pd.read_csv('fra.txt', sep='\t', names=['english', 'french', 'attribution'])
train = [
    (en, fr) for en, fr in zip(df['english'], df['french'])
]
train, valid = train_test_split(train, test_size=0.1, random_state=0)
print(len(train))

en_tokenizer, fr_tokenizer = get_tokenizer('spacy', language='en'), get_tokenizer('spacy', language='fr')

SPECIALS = ['<unk>', '<pad>', '<bos>', '<eos>']

--2023-03-08 20:47:31--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7155035 (6.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.82M  31.9MB/s    in 0.2s    

2023-03-08 20:47:32 (31.9 MB/s) - ‘fra-eng.zip’ saved [7155035/7155035]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 
188015


/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "fr" could not be loaded, trying "fr_core_news_sm" instead
  warnings.warn(


## Datasets

Functions and classes to build the vocabularies and the torch datasets.
The vocabulary is an object able to transform a string token into the id (an int) of that token in the vocabulary. 

In [ ]:
class TranslationDataset(Dataset):
    def __init__(
            self,
            dataset: list,
            en_vocab: Vocab,
            fr_vocab: Vocab,
            en_tokenizer,
            fr_tokenizer,
        ):
        super().__init__()

        self.dataset = dataset
        self.en_vocab = en_vocab
        self.fr_vocab = fr_vocab
        self.en_tokenizer = en_tokenizer
        self.fr_tokenizer = fr_tokenizer
    
    def __len__(self):
        """Return the number of examples in the dataset.
        """
        return len(self.dataset)

    def __getitem__(self, index: int) -> tuple:
        """Return a sample.

        Args
        ----
            index: Index of the sample.

        Output
        ------
            en_tokens: English tokens of the sample, as a LongTensor.
            fr_tokens: French tokens of the sample, as a LongTensor.
        """
        # Get the strings
        en_sentence, fr_sentence = self.dataset[index]

        # To list of words
        # We also add the beggining-of-sentence and end-of-sentence tokens
        en_tokens = ['<bos>'] + self.en_tokenizer(en_sentence) + ['<eos>']
        fr_tokens = ['<bos>'] + self.fr_tokenizer(fr_sentence) + ['<eos>']

        # To list of tokens
        en_tokens = self.en_vocab(en_tokens)  # list[int]
        fr_tokens = self.fr_vocab(fr_tokens)

        return torch.LongTensor(en_tokens), torch.LongTensor(fr_tokens)


def yield_tokens(dataset, tokenizer, lang):
    """Tokenize the whole dataset and yield the tokens.
    """
    assert lang in ('en', 'fr')
    sentence_idx = 0 if lang == 'en' else 1

    for sentences in dataset:
        sentence = sentences[sentence_idx]
        tokens = tokenizer(sentence)
        yield tokens


def build_vocab(dataset: list, en_tokenizer, fr_tokenizer, min_freq: int):
    """Return two vocabularies, one for each language.
    """
    en_vocab = build_vocab_from_iterator(
        yield_tokens(dataset, en_tokenizer, 'en'),
        min_freq=min_freq,
        specials=SPECIALS,
    )
    en_vocab.set_default_index(en_vocab['<unk>'])  # Default token for unknown words

    fr_vocab = build_vocab_from_iterator(
        yield_tokens(dataset, fr_tokenizer, 'fr'),
        min_freq=min_freq,
        specials=SPECIALS,
    )
    fr_vocab.set_default_index(fr_vocab['<unk>'])

    return en_vocab, fr_vocab


def preprocess(
        dataset: list,
        en_tokenizer,
        fr_tokenizer,
        max_words: int,
    ) -> list:
    """Preprocess the dataset.
    Remove samples where at least one of the sentences are too long.
    Those samples takes too much memory.
    Also remove the pending '\n' at the end of sentences.
    """
    filtered = []

    for en_s, fr_s in dataset:
        if len(en_tokenizer(en_s)) >= max_words or len(fr_tokenizer(fr_s)) >= max_words:
            continue
        
        en_s = en_s.replace('\n', '')
        fr_s = fr_s.replace('\n', '')

        filtered.append((en_s, fr_s))

    return filtered


def build_datasets(
        max_sequence_length: int,
        min_token_freq: int,
        en_tokenizer,
        fr_tokenizer,
        train: list,
        val: list,
    ) -> tuple:
    """Build the training, validation and testing datasets.
    It takes care of the vocabulary creation.

    Args
    ----
        - max_sequence_length: Maximum number of tokens in each sequences.
            Having big sequences increases dramatically the VRAM taken during training.
        - min_token_freq: Minimum number of occurences each token must have
            to be saved in the vocabulary. Reducing this number increases
            the vocabularies's size.
        - en_tokenizer: Tokenizer for the english sentences.
        - fr_tokenizer: Tokenizer for the french sentences.
        - train and val: List containing the pairs (english, french) sentences.


    Output
    ------
        - (train_dataset, val_dataset): Tuple of the two TranslationDataset objects.
    """
    datasets = [
        preprocess(samples, en_tokenizer, fr_tokenizer, max_sequence_length)
        for samples in [train, val]
    ]

    en_vocab, fr_vocab = build_vocab(datasets[0], en_tokenizer, fr_tokenizer, min_token_freq)

    datasets = [
        TranslationDataset(samples, en_vocab, fr_vocab, en_tokenizer, fr_tokenizer)
        for samples in datasets
    ]

    return datasets

In [ ]:
def generate_batch(data_batch: list, src_pad_idx: int, tgt_pad_idx: int) -> tuple:
    """Add padding to the given batch so that all
    the samples are of the same size.

    Args
    ----
        data_batch: List of samples.
            Each sample is a tuple of LongTensors of varying size.
        src_pad_idx: Source padding index value.
        tgt_pad_idx: Target padding index value.
    
    Output
    ------
        en_batch: Batch of tokens for the padded english sentences.
            Shape of [batch_size, max_en_len].
        fr_batch: Batch of tokens for the padded french sentences.
            Shape of [batch_size, max_fr_len].
    """
    en_batch, fr_batch = [], []
    for en_tokens, fr_tokens in data_batch:
        en_batch.append(en_tokens)
        fr_batch.append(fr_tokens)

    en_batch = pad_sequence(en_batch, padding_value=src_pad_idx, batch_first=True)
    fr_batch = pad_sequence(fr_batch, padding_value=tgt_pad_idx, batch_first=True)
    return en_batch, fr_batch

# Models architecture
This is where you have to code the architectures.

In a machine translation task, the model takes as input the whole
source sentence along with the current known tokens of the target,
and predict the next token in the target sequence.
This means that the target tokens are predicted in an autoregressive
manner, starting from the first token (right after the *\<bos\>* token) and producing tokens one by one until the last *\<eos\>* token.

Formally, we define $s = [s_1, ..., s_{N_s}]$ as the source sequence made of $N_s$ tokens.
We also define $t^i = [t_1, ..., t_i]$ as the target sequence at the beginning of the step $i$.

The output of the model parameterized by $\theta$ is:

$$
T_{i+1} = p(t_{i+1} | s, t^i ; \theta )
$$

Where $T_{i+1}$ is the distribution of the next token $t_{i+1}$.

The loss is simply a *cross entropy loss* over the whole steps, where each class is a token of the vocabulary.

![RNN schema for machinea translation](https://www.simplilearn.com/ice9/free_resources_article_thumb/machine-translation-model-with-encoder-decoder-rnn.jpg)

Note that in this image the english sentence is provided in reverse. 

---

In pytorch, there is no dinstinction between an intermediate layer or a whole model having multiple layers in itself.
Every layers or models inherit from the `torch.nn.Module`.
This module needs to define the `__init__` method where you instanciate the layers,
and the `forward` method where you decide how the inputs and the layers of the module interact between them.
Thanks to the autograd computations of pytorch, you do not have
to implement any backward method!

A really important advice is to **always look at
the shape of your input and your output.**
From that, you can often guess how the layers should interact
with the inputs to produce the right output.
You can also easily detect if there's something wrong going on.

You are more than advised to use the `einops` library and the `torch.einsum` function. This will require less operations than 'classical' code, but note that it's a bit trickier to use.
This is a way of describing tensors manipulation with strings, bypassing the multiple tensor methods executed in the background.
You can find a nice presentation of `einops` [here](https://einops.rocks/1-einops-basics/).
A paper has just been released about einops [here](https://paperswithcode.com/paper/einops-clear-and-reliable-tensor).

**A great tutorial on pytorch can be found [here](https://stanford.edu/class/cs224n/materials/CS224N_PyTorch_Tutorial.html).**
Spending 3 hours on this tutorial is *no* waste of time.

## RNN models

### RNN
Here you have to implement a recurrent neural network. You will need to create a single RNN Layer, and a module allowing to stack these layers. Look up the pytorch documentation to figure out this module's operations and what is communicated from one layer to another.

The `RNNCell` layer produce one hidden state vector for each sentence in the batch
(useful for the output of the encoder), and also produce one embedding for each
token in each sentence (useful for the output of the decoder).

The `RNN` module is composed of a stack of `RNNCell`. Each token embeddings
coming out from a previous `RNNCell` is used as an input for the next `RNNCell` layer.

**Be careful !** Our `RNNCell` implementation is not exactly the same thing as
the PyTorch's `nn.RNNCell`. PyTorch implements only the operations for one token
(so you would need to loop through each tokens inside the `RNN` instead).
You are free to implement `RNN` and `RNNCell` the way you want, as long as it has the expected behaviour of a RNN.

The same thing apply for the `GRU` and `GRUCell`.


In [ ]:
class RNNCell(nn.Module):
    """A single RNN layer.
    
    Parameters
    ----------
        input_size: Size of each input token.
        hidden_size: Size of each RNN hidden state.
        dropout: Dropout rate.

    Important note: This layer does not exactly the same thing as nn.RNNCell does.
    PyTorch implementation is only doing one simple pass over one token for each batch.
    This implementation is taking the whole sequence of each batch and provide the
    final hidden state along with the embeddings of each token in each sequence.
    """
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            dropout: float,
        ):
        super().__init__()

        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.input_to_hidden = nn.Linear(input_size, hidden_size)
        self.hidden_to_hidden = nn.Linear(hidden_size, hidden_size)

        # Activation and layers
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)

        # device
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def forward(self, X: torch.FloatTensor, h: torch.FloatTensor) -> tuple:
        """Go through all the sequence in x, iteratively updating
        the hidden state h.

        Args
        ----
            x: Input sequence.
                Shape of [batch_size, seq_len, input_size].
            h: Initial hidden state.
                Shape of [batch_size, hidden_size].

        Output
        ------
            y: Token embeddings.
                Shape of [batch_size, seq_len, hidden_size].
            h: Last hidden state.
                Shape of [batch_size, hidden_size].
        """
        batch_size = X.shape[0]
        seq_len = X.shape[1]

        y = torch.zeros(batch_size, seq_len, self.hidden_size).to(self.device)
        for t in range(seq_len):
            a = self.input_to_hidden(X[:, t, :]) + self.hidden_to_hidden(h)
            h = self.dropout(self.tanh(a))
            y[:, t, :] = h
        return y, h


class RNN(nn.Module):
    """Implementation of an RNN based
    on https://pytorch.org/docs/stable/generated/torch.nn.RNN.html.

    Parameters
    ----------
        input_size: Size of each input token.
        hidden_size: Size of each RNN hidden state.
        num_layers: Number of layers (RNNCell or GRUCell).
        dropout: Dropout rate.
        model_type: Either 'RNN' or 'GRU', to select which model we want.
            This parameter can be removed if you decide to use the module `GRU`.
            Indeed, `GRU` should have exactly the same code as this module,
            but with `GRUCell` instead of `RNNCell`. We let the freedom for you
            to decide at which level you want to specialise the modules (either
            in `TranslationRNN` by creating a `GRU` or a `RNN`, or in `RNN`
            by creating a `GRUCell` or a `RNNCell`).
    """
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            num_layers: int,
            dropout: float,
            model_type: str,
        ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers


        if model_type == "RNN":
            self.layers = nn.ModuleList([RNNCell(input_size, hidden_size, dropout)]\
                        + [RNNCell(hidden_size, hidden_size, dropout) for _ in range(num_layers-1)])
        elif model_type == "GRU":
            self.layers =nn.ModuleList([GRUCell(input_size, hidden_size, dropout)]\
                    + [GRUCell(hidden_size, hidden_size, dropout) for _ in range(num_layers-1)])
        else:
            raise ValueError(f"{model_type} is invalid. Use either 'RNN' or 'GRU'.")

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def forward(self, X: torch.FloatTensor, hidden: torch.FloatTensor=None) -> tuple:
        """Pass the input sequence through all the RNN cells.
        Returns the output and the final hidden state of each RNN layer

        Args
        ----
            x: Input sequence.
                Shape of [batch_size, seq_len, input_size].
            h: Hidden state for each RNN layer.
                Can be None, in which case an initial hidden state is created.
                Shape of [batch_size, n_layers, hidden_size].

        Output
        ------
            y: Output embeddings for each token after the RNN layers.
                Shape of [batch_size, seq_len, hidden_size].
            h: Final hidden state.
                Shape of [batch_size, n_layers, hidden_size].
        """
        batch_size = X.shape[0]
        seq_len = X.shape[1]
        if hidden is None:
            hidden = torch.zeros(batch_size, self.num_layers, self.hidden_size).to(self.device)
        
        H = torch.zeros_like(hidden).to(self.device)
        
        for layer in range(self.num_layers):
            h_n = hidden[:, layer, :]
            X, h_n = self.layers[layer](X, h_n)
            H[:, layer, :] = h_n
        
        return X, H


test_model = RNN(30, 20, 3, 0.5, "RNN")
dict(test_model.named_parameters()).keys()

dict_keys(['layers.0.input_to_hidden.weight', 'layers.0.input_to_hidden.bias', 'layers.0.hidden_to_hidden.weight', 'layers.0.hidden_to_hidden.bias', 'layers.1.input_to_hidden.weight', 'layers.1.input_to_hidden.bias', 'layers.1.hidden_to_hidden.weight', 'layers.1.hidden_to_hidden.bias', 'layers.2.input_to_hidden.weight', 'layers.2.input_to_hidden.bias', 'layers.2.hidden_to_hidden.weight', 'layers.2.hidden_to_hidden.bias'])

### GRU
Here you have to implement a GRU-RNN. This architecture is close to the Vanilla RNN but perform different operations. Look up the pytorch documentation to figure out the differences.

In [ ]:
class GRU(nn.Module):
    """Implementation of a GRU based on https://pytorch.org/docs/stable/generated/torch.nn.GRU.html.

    Parameters
    ----------
        input_size: Size of each input token.
        hidden_size: Size of each RNN hidden state.
        num_layers: Number of layers.
        dropout: Dropout rate.
    """
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            num_layers: int,
            dropout: float,
        ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.layers =nn.ModuleList([GRUCell(input_size, hidden_size, dropout)]\
                    + [GRUCell(hidden_size, hidden_size, dropout) for _ in range(num_layers-1)])
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'


    def forward(self, x: torch.FloatTensor, hidden: torch.FloatTensor=None) -> tuple:
        """
        Args
        ----
            x: Input sequence
                Shape of [batch_size, seq_len, input_size].
            h: Initial hidden state for each layer.
                If 'None', then an initial hidden state (a zero filled tensor)
                is created.
                Shape of [batch_size, n_layers, hidden_size].

        Output
        ------
            output:
                Shape of [batch_size, seq_len, hidden_size].
            h_n: Final hidden state.
                Shape of [batch_size, n_layers, hidden size].
        """
        batch_size = X.shape[0]
        seq_len = X.shape[1]
        if hidden is None:
            hidden = torch.zeros(batch_size, self.num_layers, self.hidden_size).to(self.device)
        H = torch.zeros_like(hidden).to(self.device)
        
        for layer in range(self.num_layers):
            h_n = hidden[:, layer, :]
            X, h_n = self.layers[layer](X, h_n)
            H[:, layer, :] = h_n
        
        return X, H


class GRUCell(nn.Module):
    """A single GRU layer.
    
    Parameters
    ----------
        input_size: Size of each input token.
        hidden_size: Size of each RNN hidden state.
        dropout: Dropout rate.
    """
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            dropout: float,
        ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Layers
        self.input_to_r = nn.Linear(input_size, hidden_size)
        self.hidden_to_r = nn.Linear(hidden_size, hidden_size)

        self.input_to_z = nn.Linear(input_size, hidden_size)
        self.hidden_to_z = nn.Linear(hidden_size, hidden_size)

        self.input_to_n = nn.Linear(input_size, hidden_size)
        self.hidden_to_n = nn.Linear(hidden_size, hidden_size)

        # Activation Layers
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def forward(self, X: torch.FloatTensor, h: torch.FloatTensor) -> tuple:
        """
        Args
        ----
            x: Input sequence.
                Shape of [batch_size, seq_len, input_size].
            h: Initial hidden state.
                Shape of [batch_size, hidden_size].

        Output
        ------
            y: Token embeddings.
                Shape of [batch_size, seq_len, hidden_size].
            h: Last hidden state.
                Shape of [batch_size, hidden_size].
        """
        batch_size = X.shape[0]
        seq_len = X.shape[1]

        y = torch.zeros(batch_size, seq_len, self.hidden_size).to(self.device)
        for t in range(seq_len):
            r = self.dropout(self.sigmoid(self.input_to_r(X[:, t, :]) + self.hidden_to_r(h)))
            z = self.dropout(self.sigmoid(self.input_to_z(X[:, t, :]) + self.hidden_to_z(h)))
            n = self.dropout(self.tanh(self.input_to_n(X[:, t, :]) + r*(self.hidden_to_n(h))))
            ones_z = torch.ones_like(z).to(self.device)
            h = (ones_z - z)*n + z*h
            y[:, t, :] = h
        return y, h

test_model = GRU(3, 40, 3, 0.5)
dict(test_model.named_parameters()).keys()

dict_keys(['layers.0.input_to_r.weight', 'layers.0.input_to_r.bias', 'layers.0.hidden_to_r.weight', 'layers.0.hidden_to_r.bias', 'layers.0.input_to_z.weight', 'layers.0.input_to_z.bias', 'layers.0.hidden_to_z.weight', 'layers.0.hidden_to_z.bias', 'layers.0.input_to_n.weight', 'layers.0.input_to_n.bias', 'layers.0.hidden_to_n.weight', 'layers.0.hidden_to_n.bias', 'layers.1.input_to_r.weight', 'layers.1.input_to_r.bias', 'layers.1.hidden_to_r.weight', 'layers.1.hidden_to_r.bias', 'layers.1.input_to_z.weight', 'layers.1.input_to_z.bias', 'layers.1.hidden_to_z.weight', 'layers.1.hidden_to_z.bias', 'layers.1.input_to_n.weight', 'layers.1.input_to_n.bias', 'layers.1.hidden_to_n.weight', 'layers.1.hidden_to_n.bias', 'layers.2.input_to_r.weight', 'layers.2.input_to_r.bias', 'layers.2.hidden_to_r.weight', 'layers.2.hidden_to_r.bias', 'layers.2.input_to_z.weight', 'layers.2.input_to_z.bias', 'layers.2.hidden_to_z.weight', 'layers.2.hidden_to_z.bias', 'layers.2.input_to_n.weight', 'layers.2.input

### Translation RNN

This module instanciates a vanilla RNN or a GRU-RNN and performs the translation task. You have to:
* Encode the source and target sequence
* Pass the final hidden state of the encoder to the decoder (one for each layer)
* Decode the hidden state into the target sequence

We use teacher forcing for training, meaning that when the next token is predicted, that prediction is based on the previous true target tokens. 

In [ ]:
class TranslationRNN(nn.Module):
    """Basic RNN encoder and decoder for a translation task.
    It can run as a vanilla RNN or a GRU-RNN.

    Parameters
    ----------
        n_tokens_src: Number of tokens in the source vocabulary.
        n_tokens_tgt: Number of tokens in the target vocabulary.
        dim_embedding: Dimension size of the word embeddings (for both language).
        dim_hidden: Dimension size of the hidden layers in the RNNs
            (for both the encoder and the decoder).
        n_layers: Number of layers in the RNNs.
        dropout: Dropout rate.
        src_pad_idx: Source padding index value.
        tgt_pad_idx: Target padding index value.
        model_type: Either 'RNN' or 'GRU', to select which model we want.
    """

    def __init__(
            self,
            n_tokens_src: int,
            n_tokens_tgt: int,
            dim_embedding: int,
            dim_hidden: int,
            n_layers: int,
            dropout: float,
            src_pad_idx: int,
            tgt_pad_idx: int,
            model_type: str,
        ):
        super().__init__()
        # params
        self.n_tokens_src = n_tokens_src
        self.n_tokens_tgt = n_tokens_tgt
        self.dim_embedding = dim_embedding
        self.dim_hidden = dim_hidden
        self.n_layers = n_layers
        self.dropout = dropout
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx
        self.model_type = model_type

        # layers
        self.src_embedding = nn.Embedding(n_tokens_src, dim_embedding, src_pad_idx)
        self.tgt_embedding = nn.Embedding(n_tokens_tgt, dim_embedding, tgt_pad_idx)
        self.encoder = RNN(dim_embedding, dim_hidden, n_layers, dropout, model_type)
        self.decoder = RNN(dim_embedding, dim_hidden, n_layers, dropout, model_type)
        self.fc_out = nn.Sequential(
            nn.Linear(dim_hidden, self.dim_embedding),
            nn.LeakyReLU(),
            nn.LayerNorm(self.dim_embedding),
            nn.Linear(self.dim_embedding, self.dim_embedding),
            nn.LeakyReLU(),
            nn.LayerNorm(self.dim_embedding),
            nn.Linear(self.dim_embedding, self.dim_embedding),
            nn.LeakyReLU(),
            nn.LayerNorm(self.dim_embedding),
            nn.Linear(self.dim_embedding, n_tokens_tgt)
        )


    def forward(
        self,
        source: torch.LongTensor,
        target: torch.LongTensor
    ) -> torch.FloatTensor:
        """Predict the target tokens logites based on the source tokens.

        Args
        ----
            source: Batch of source sentences.
                Shape of [batch_size, src_seq_len].
            target: Batch of target sentences.
                Shape of [batch_size, tgt_seq_len].
        
        Output
        ------
            y: Distributions over the next token for all tokens in each sentences.
                Those need to be the logits only, do not apply a softmax because
                it will be done in the loss computation for numerical stability.
                See https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html for more informations.
                Shape of [batch_size, tgt_seq_len, n_tokens_tgt].
        """
        tgt_seq_len = target.shape[1]
        batch_size = target.shape[0]

        src_embed = self.src_embedding(source)
        tgt_embed = self.tgt_embedding(target)


        _, hidden = self.encoder(src_embed)
        output, _ = self.decoder(tgt_embed, hidden)
        prediction = self.fc_out(output)
        # print("prediction.shape ", prediction.shape)
        return prediction


test_model = TranslationRNN(10000, 12000, 40, 30, 3, 0.5, 30, 30, 'RNN')
dict(test_model.named_parameters()).keys()

dict_keys(['src_embedding.weight', 'tgt_embedding.weight', 'encoder.layers.0.input_to_hidden.weight', 'encoder.layers.0.input_to_hidden.bias', 'encoder.layers.0.hidden_to_hidden.weight', 'encoder.layers.0.hidden_to_hidden.bias', 'encoder.layers.1.input_to_hidden.weight', 'encoder.layers.1.input_to_hidden.bias', 'encoder.layers.1.hidden_to_hidden.weight', 'encoder.layers.1.hidden_to_hidden.bias', 'encoder.layers.2.input_to_hidden.weight', 'encoder.layers.2.input_to_hidden.bias', 'encoder.layers.2.hidden_to_hidden.weight', 'encoder.layers.2.hidden_to_hidden.bias', 'decoder.layers.0.input_to_hidden.weight', 'decoder.layers.0.input_to_hidden.bias', 'decoder.layers.0.hidden_to_hidden.weight', 'decoder.layers.0.hidden_to_hidden.bias', 'decoder.layers.1.input_to_hidden.weight', 'decoder.layers.1.input_to_hidden.bias', 'decoder.layers.1.hidden_to_hidden.weight', 'decoder.layers.1.hidden_to_hidden.bias', 'decoder.layers.2.input_to_hidden.weight', 'decoder.layers.2.input_to_hidden.bias', 'decode

## Transformer model
Here you have to code the Transformer architecture.
It is divided in three parts:
* Attention layers
* Encoder and decoder layers
* Main layers (gather the encoder and decoder layers)

The [illustrated transformer](https://jalammar.github.io/illustrated-transformer/) blog can help you
understanding how the architecture works.
Once this is done, you can use [the annontated transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html) to have an idea of how to code this architecture.
We encourage you to use `torch.einsum` and the `einops` library as much as you can. It will make your code simpler.

---
**Implementation order**

To help you with the implementation, we advise you following this order:
* Implement `TranslationTransformer` and use `nn.Transformer` instead of `Transformer`
* Implement `Transformer` and use `nn.TransformerDecoder` and `nn.TransformerEnocder`
* Implement the `TransformerDecoder` and `TransformerEncoder` and use `nn.MultiHeadAttention`
* Implement `MultiHeadAttention`

Do not forget to add `batch_first=True` when necessary in the `nn` modules.

### Attention layers
We use a `MultiHeadAttention` module, that is able to perform self-attention aswell as cross-attention (depending on what you give as queries, keys and values).

**Attention**


It takes the multiheaded queries, keys and values as input.
It computes the attention between the queries and the keys and return the attended values.

The implementation of this function can greatly be improved with *einsums*.

**MultiheadAttention**

Computes the multihead queries, keys and values and feed them to the `attention` function.
You also need to merge the key padding mask and the attention mask into one mask.

The implementation of this module can greatly be improved with *einops.rearrange*.

In [ ]:
def attention(
        q: torch.FloatTensor,
        k: torch.FloatTensor,
        v: torch.FloatTensor,
        mask: torch.BoolTensor=None,
        dropout: nn.Dropout=None,
    ) -> tuple:
    """Computes multihead scaled dot-product attention from the
    projected queries, keys and values.

    Args
    ----
        q: Batch of queries.
            Shape of [batch_size, seq_len_1, n_heads, dim_model].
        k: Batch of keys.
            Shape of [batch_size, seq_len_2, n_heads, dim_model].
        v: Batch of values.
            Shape of [batch_size, seq_len_2, n_heads, dim_model].
        mask: Prevent tokens to attend to some other tokens (for padding or autoregressive attention).
            Attention is prevented where the mask is `True`.
            Shape of [batch_size, n_heads, seq_len_1, seq_len_2],
            or broadcastable to that shape.
        dropout: Dropout layer to use.

    Output
    ------
        y: Multihead scaled dot-attention between the queries, keys and values.
            Shape of [batch_size, seq_len_1, n_heads, dim_model].
        attn: Computed attention between the keys and the queries.
            Shape of [batch_size, n_heads, seq_len_1, seq_len_2].
    """
    attn = torch.einsum("bshk,blhk->bhsl", [q, k]) 
    if mask is not None:
        attn = attn.masked_fill(mask == 0, -np.inf)
    p_attn = torch.softmax(attn / np.sqrt(k.shape[-1]), dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    y = torch.einsum("bnsl,blnd->bsnd", [p_attn, v])
    return y, attn

In [ ]:
class MultiheadAttention(nn.Module):
    """Multihead attention module.
    Can be used as a self-attention and cross-attention layer.
    The queries, keys and values are projected into multiple heads
    before computing the attention between those tensors.

    Parameters
    ----------
        dim: Dimension of the input tokens.
        n_heads: Number of heads. `dim` must be divisible by `n_heads`.
        dropout: Dropout rate.
    """
    def __init__(
            self,
            dim: int,
            n_heads: int,
            dropout: float,
        ):
        super().__init__()

        assert dim % n_heads == 0, "'dim' and 'n_heads' must be divisible"
        self.dim = dim
        self.d_k = dim // n_heads
        self.n_heads = n_heads
        self.w_qs = nn.Linear(self.dim, self.dim)
        self.w_ks = nn.Linear(self.dim, self.dim)
        self.w_vs = nn.Linear(self.dim, self.dim)
        self.w_out = nn.Linear(self.d_k*self.n_heads, dim)
        self.dropout = nn.Dropout(dropout)
        

    def forward(
            self,
            q: torch.FloatTensor,
            k: torch.FloatTensor,
            v: torch.FloatTensor,
            key_padding_mask: torch.BoolTensor = None,
            attn_mask: torch.BoolTensor = None,
        ) -> torch.FloatTensor:
        """Computes the scaled multi-head attention form the input queries,
        keys and values.

        Project those queries, keys and values before feeding them
        to the `attention` function.

        The masks are boolean masks. Tokens are prevented to attends to
        positions where the mask is `True`.

        Args
        ----
            q: Batch of queries.
                Shape of [batch_size, seq_len_1, dim_model].
            k: Batch of keys.
                Shape of [batch_size, seq_len_2, dim_model].
            v: Batch of values.
                Shape of [batch_size, seq_len_2, dim_model].
            key_padding_mask: Prevent attending to padding tokens.
                Shape of [batch_size, seq_len_2].
            attn_mask: Prevent attending to subsequent tokens.
                Shape of [seq_len_1, seq_len_2].

        Output
        ------
            y: Computed multihead attention.
                Shape of [batch_size, seq_len_1, dim_model].
        """
        if attn_mask is not None:
            # Same attn_mask applied to all h heads.
            attn_mask = einops.repeat(attn_mask, "l t -> b h l t", b=q.shape[0], h=self.n_heads)

        q = einops.rearrange(self.w_qs(q), 'b l (head k) -> b l head k', head=self.n_heads, k=self.d_k)
        k = einops.rearrange(self.w_ks(k), 'b t (head k) -> b t head k', head=self.n_heads, k=self.d_k)
        v = einops.rearrange(self.w_vs(v), 'b t (head v) -> b t head v', head=self.n_heads, v=self.d_k)

        x, attn = attention(q, k, v, attn_mask, self.dropout)
        x = einops.rearrange(x, "b l head d_k -> b l (head d_k)", head=self.n_heads, d_k=self.d_k)
        y = self.dropout(self.w_out(x))
        return y

### Encoder and decoder layers

**TranformerEncoder**

Apply self-attention layers onto the source tokens.
It only needs the source key padding mask.


**TranformerDecoder**

Apply masked self-attention layers to the target tokens and cross-attention
layers between the source and the target tokens.
It needs the source and target key padding masks, and the target attention mask.

In [ ]:
class TransformerDecoderLayer(nn.Module):
    """Single decoder layer.

    Parameters
    ----------
        d_model: The dimension of decoders inputs/outputs.
        dim_feedforward: Hidden dimension of the feedforward networks.
        nheads: Number of heads for each multi-head attention.
        dropout: Dropout rate.
    """

    def __init__(
            self,
            d_model: int,
            d_ff: int,
            nhead: int,
            dropout: float
        ):
        super().__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.nhead = nhead
        self.dropout = nn.Dropout(dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

        self.masked_attention = MultiheadAttention(d_model, nhead, dropout)
        self.attention = MultiheadAttention(d_model, nhead, dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_model*d_ff),
            nn.ReLU(),
            nn.Linear(d_model*d_ff, d_model)
        )

        # TODO

    def forward(
            self,
            src: torch.FloatTensor,
            tgt: torch.FloatTensor,
            tgt_mask_attn: torch.BoolTensor,
            src_key_padding_mask: torch.BoolTensor=None,
            tgt_key_padding_mask: torch.BoolTensor=None,
        ) -> torch.FloatTensor:
        """Decode the next target tokens based on the previous tokens.

        Args
        ----
            src: Batch of source sentences.
                Shape of [batch_size, src_seq_len, dim_model].
            tgt: Batch of target sentences.
                Shape of [batch_size, tgt_seq_len, dim_model].
            tgt_mask_attn: Mask to prevent attention to subsequent tokens.
                Shape of [tgt_seq_len, tgt_seq_len].
            src_key_padding_mask: Mask to prevent attention to padding in src sequence.
                Shape of [batch_size, src_seq_len].
            tgt_key_padding_mask: Mask to prevent attention to padding in tgt sequence.
                Shape of [batch_size, tgt_seq_len].

        Output
        ------
            y:  Batch of sequence of embeddings representing the predicted target tokens
                Shape of [batch_size, tgt_seq_len, dim_model].
        """
        masked_attn = self.masked_attention(tgt, tgt, tgt, attn_mask=tgt_mask_attn)
        x = self.norm1(self.dropout(masked_attn + tgt))
        multiheaded_attn = self.attention(x, src, src)
        x = self.norm2(self.dropout(multiheaded_attn + x))
        y = self.dropout(self.feed_forward(x))
        y = self.norm3(y + x)
        return y


class TransformerDecoder(nn.Module):
    """Implementation of the transformer decoder stack.

    Parameters
    ----------
        d_model: The dimension of decoders inputs/outputs.
        dim_feedforward: Hidden dimension of the feedforward networks.
        num_decoder_layers: Number of stacked decoders.
        nheads: Number of heads for each multi-head attention.
        dropout: Dropout rate.
    """

    def __init__(
            self,
            d_model: int,
            d_ff: int,
            num_decoder_layer:int ,
            nhead: int,
            dropout: float
        ):
        super().__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.num_decoder_layer = num_decoder_layer
        self.nhead = nhead
        self.dropout = dropout
        self.layers = nn.ModuleList(
            [
                TransformerDecoderLayer(
                    d_model, d_ff, nhead, dropout
                ) for _ in range(num_decoder_layer)
            ]
        )

        

    def forward(
            self,
            src: torch.FloatTensor,
            tgt: torch.FloatTensor,
            tgt_mask_attn: torch.BoolTensor=None,
            src_key_padding_mask: torch.BoolTensor=None,
            tgt_key_padding_mask: torch.BoolTensor=None,
        ) -> torch.FloatTensor:
        """Decodes the source sequence by sequentially passing.
        the encoded source sequence and the target sequence through the decoder stack.

        Args
        ----
            src: Batch of encoded source sentences.
                Shape of [batch_size, src_seq_len, dim_model].
            tgt: Batch of taget sentences.
                Shape of [batch_size, tgt_seq_len, dim_model].
            tgt_mask_attn: Mask to prevent attention to subsequent tokens.
                Shape of [tgt_seq_len, tgt_seq_len].
            src_key_padding_mask: Mask to prevent attention to padding in src sequence.
                Shape of [batch_size, src_seq_len].
            tgt_key_padding_mask: Mask to prevent attention to padding in tgt sequence.
                Shape of [batch_size, tgt_seq_len].

        Output
        ------
            y:  Batch of sequence of embeddings representing the predicted target tokens
                Shape of [batch_size, tgt_seq_len, dim_model].
        """
        out = tgt
        for i, layer in enumerate(self.layers):
            out = layer(src, out, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask)
        return out


class TransformerEncoderLayer(nn.Module):
    """Single encoder layer.

    Parameters
    ----------
        d_model: The dimension of input tokens.
        dim_feedforward: Hidden dimension of the feedforward networks.
        nheads: Number of heads for each multi-head attention.
        dropout: Dropout rate.
    """

    def __init__(
            self,
            d_model: int,
            d_ff: int,
            nhead: int,
            dropout: float,
        ):
        super().__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_head = nhead
        self.dropout = nn.Dropout(dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        self.attention = MultiheadAttention(d_model, nhead, dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_model*d_ff),
            nn.ReLU(),
            nn.Linear(d_model*d_ff, d_model)
        )

    def forward(
        self,
        src: torch.FloatTensor,
        key_padding_mask: torch.BoolTensor = None
        ) -> torch.FloatTensor:
        """Encodes the input. Does not attend to masked inputs.

        Args
        ----
            src: Batch of embedded source tokens.
                Shape of [batch_size, src_seq_len, dim_model].
            key_padding_mask: Mask preventing attention to padding tokens.
                Shape of [batch_size, src_seq_len].

        Output
        ------
            y: Batch of encoded source tokens.
                Shape of [batch_size, src_seq_len, dim_model].
        """
        query = src#copy(src)
        keys = src#copy.deepcopy(src)
        values = src#copy.deepcopy(src)
        attention = self.attention(query, keys, values)
        x = self.norm1(self.dropout(attention + query))
        forward = self.feed_forward(x)
        y = self.norm2(self.dropout(forward + x))
        return y


class TransformerEncoder(nn.Module):
    """Implementation of the transformer encoder stack.

    Parameters
    ----------
        d_model: The dimension of encoders inputs.
        dim_feedforward: Hidden dimension of the feedforward networks.
        num_encoder_layers: Number of stacked encoders.
        nheads: Number of heads for each multi-head attention.
        dropout: Dropout rate.
    """

    def __init__(
            self,
            d_model: int,
            dim_feedforward: int,
            num_encoder_layers: int,
            nheads: int,
            dropout: float
        ):
        super().__init__()
        self.d_model = d_model
        self.dim_ff = dim_feedforward
        self.num_encoder_layers = num_encoder_layers
        self.nheads = nheads
        self.dropout = nn.Dropout(dropout)

        self.layers = nn.ModuleList(
            [
                TransformerEncoderLayer(
                    d_model, dim_feedforward, nheads, dropout
                ) for _ in range(num_encoder_layers)
            ]
        )

    def forward(
            self,
            src: torch.FloatTensor,
            key_padding_mask: torch.BoolTensor = None
        ) -> torch.FloatTensor:
        """Encodes the source sequence by sequentially passing.
        the source sequence through the encoder stack.

        Args
        ----
            src: Batch of embedded source sentences.
                Shape of [batch_size, src_seq_len, dim_model].
            key_padding_mask: Mask preventing attention to padding tokens.
                Shape of [batch_size, src_seq_len].

        Output
        ------
            y: Batch of encoded source sequence.
                Shape of [batch_size, src_seq_len, dim_model].
        """
        
        out = src
        for layer in self.layers:
            out = layer(out, key_padding_mask)
        return out

### Main layers
This section gather the `Transformer` and the `TranslationTransformer` modules.

**Transformer**


The classical transformer architecture.
It takes the source and target tokens embeddings and
do the forward pass through the encoder and decoder.

**Translation Transformer**

Compute the source and target tokens embeddings, and apply a final head to produce next token logits.
The output must not be the softmax but just the logits, because we use the `nn.CrossEntropyLoss`.

It also creates the *src_key_padding_mask*, the *tgt_key_padding_mask* and the *tgt_mask_attn*.

In [ ]:
class Transformer(nn.Module):
    """Implementation of a Transformer based on the paper: https://arxiv.org/pdf/1706.03762.pdf.

    Parameters
    ----------
        d_model: The dimension of encoders/decoders inputs/ouputs.
        nhead: Number of heads for each multi-head attention.
        num_encoder_layers: Number of stacked encoders.
        num_decoder_layers: Number of stacked encoders.
        dim_feedforward: Hidden dimension of the feedforward networks.
        dropout: Dropout rate.
    """

    def __init__(
            self,
            d_model: int,
            nhead: int,
            num_encoder_layers: int,
            num_decoder_layers: int,
            dim_feedforward: int,
            dropout: float,
        ):
        super().__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.d_ff = dim_feedforward
        self.dropout = dropout

        self.encoder = TransformerEncoder(d_model, dim_feedforward, num_encoder_layers, nhead, dropout)
        self.decoder = TransformerDecoder(d_model, dim_feedforward, num_decoder_layers, nhead, dropout)


    def forward(
            self,
            src: torch.FloatTensor,
            tgt: torch.FloatTensor,
            tgt_mask_attn: torch.BoolTensor=None,
            src_key_padding_mask: torch.BoolTensor=None,
            tgt_key_padding_mask: torch.BoolTensor=None
        ) -> torch.FloatTensor:
        """Compute next token embeddings.

        Args
        ----
            src: Batch of source sequences.
                Shape of [batch_size, src_seq_len, dim_model].
            tgt: Batch of target sequences.
                Shape of [batch_size, tgt_seq_len, dim_model].
            tgt_mask_attn: Mask to prevent attention to subsequent tokens.
                Shape of [tgt_seq_len, tgt_seq_len].
            src_key_padding_mask: Mask to prevent attention to padding in src sequence.
                Shape of [batch_size, src_seq_len].
            tgt_key_padding_mask: Mask to prevent attention to padding in tgt sequence.
                Shape of [batch_size, tgt_seq_len].

        Output
        ------
            y: Next token embeddings, given the previous target tokens and the source tokens.
                Shape of [batch_size, tgt_seq_len, dim_model].
        """
        context = self.encoder(src, src_key_padding_mask)
        output = self.decoder(context, tgt, tgt_mask_attn, src_key_padding_mask, tgt_key_padding_mask)
        return output


class TranslationTransformer(nn.Module):
    """Basic Transformer encoder and decoder for a translation task.
    Manage the masks creation, and the token embeddings.
    Position embeddings can be learnt with a standard `nn.Embedding` layer.

    Parameters
    ----------
        n_tokens_src: Number of tokens in the source vocabulary.
        n_tokens_tgt: Number of tokens in the target vocabulary.
        n_heads: Number of heads for each multi-head attention.
        dim_embedding: Dimension size of the word embeddings (for both language).
        dim_hidden: Dimension size of the feedforward layers
            (for both the encoder and the decoder).
        n_layers: Number of layers in the encoder and decoder.
        dropout: Dropout rate.
        src_pad_idx: Source padding index value.
        tgt_pad_idx: Target padding index value.
    """
    def __init__(
            self,
            n_tokens_src: int,
            n_tokens_tgt: int,
            n_heads: int,
            dim_embedding: int,
            dim_hidden: int,
            n_layers: int,
            dropout: float,
            src_pad_idx: int,
            tgt_pad_idx: int,
        ):
        super().__init__()

        self.n_tokens_src = n_tokens_src
        self.n_tokens_tgt = n_tokens_tgt
        self.n_heads = n_heads
        self.dim_embedding = dim_embedding
        self.dim_hidden = dim_hidden
        self.n_layers = n_layers
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx
        self.max_len = 500

        self.transformer = Transformer(dim_embedding, n_heads, n_layers, n_layers, dim_hidden, dropout)
        self.src_embedding = nn.Embedding(n_tokens_src, dim_embedding, src_pad_idx)
        self.pos_src_embedding = nn.Embedding(self.max_len, dim_embedding)
        
        self.tgt_embedding = nn.Embedding(n_tokens_tgt, dim_embedding, tgt_pad_idx)
        self.pos_tgt_embedding = nn.Embedding(self.max_len, dim_embedding)

        self.fc_out = nn.Sequential(
            nn.Linear(dim_embedding, dim_hidden),
            nn.LayerNorm(dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.LayerNorm(dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, n_tokens_tgt)
        )

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'


    def forward(
            self,
            source: torch.LongTensor,
            target: torch.LongTensor
        ) -> torch.FloatTensor:
        """Predict the target tokens logites based on the source tokens.

        Args
        ----
            source: Batch of source sentences.
                Shape of [batch_size, seq_len_src].
            target: Batch of target sentences.
                Shape of [batch_size, seq_len_tgt].

        Output
        ------
            y: Distributions over the next token for all tokens in each sentences.
                Those need to be the logits only, do not apply a softmax because
                it will be done in the loss computation for numerical stability.
                See https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html for more informations.
                Shape of [batch_size, seq_len_tgt, n_tokens_tgt].
        """
        N, seq_len_src = source.shape
        _, seq_len_tgt = target.shape
        src_positions = torch.arange(0, seq_len_src).expand(N, seq_len_src).to(self.device)
        tgt_positions = torch.arange(0, seq_len_tgt).expand(N, seq_len_tgt).to(self.device)

        src = self.dropout(self.src_embedding(source) + self.pos_src_embedding(src_positions))
        tgt = self.dropout(self.tgt_embedding(target) + self.pos_tgt_embedding(tgt_positions))
        tgt_attn_mask = self.make_trg_mask(target)

        decoded = self.transformer(src, tgt, tgt_attn_mask)
        output = self.fc_out(decoded)
        return output

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len)))

        return trg_mask.to(self.device)

# Greedy search

Here you have to implement a geedy search to generate a target translation from a trained model and an input source string.
The next token will simply be the most probable one.

In [ ]:
def greedy_search(
        model: nn.Module,
        source: str,
        src_vocab: Vocab,
        tgt_vocab: Vocab,
        src_tokenizer,
        device: str,
        max_sentence_length: int,
    ) -> str:
    """Do a beam search to produce probable translations.

    Args
    ----
        model: The translation model. Assumes it produces logits score (before softmax).
        source: The sentence to translate.
        src_vocab: The source vocabulary.
        tgt_vocab: The target vocabulary.
        device: Device to which we make the inference.
        max_target: Maximum number of target sentences we keep at the end of each stage.
        max_sentence_length: Maximum number of tokens for the translated sentence.

    Output
    ------
        sentence: The translated source sentence.
    """
    src_tokens = ['<bos>'] + src_tokenizer(source) + ['<eos>']
    src_tokens = src_vocab(src_tokens)

    tgt_tokens = ['<bos>']
    tgt_tokens = tgt_vocab(tgt_tokens)

    # To tensor and add unitary batch dimension
    src_tokens = torch.LongTensor(src_tokens).to(device)
    tgt_tokens = torch.LongTensor(tgt_tokens).unsqueeze(dim=0).to(device)
    target_probs = torch.FloatTensor([1]).to(device)
    model.to(device)

    EOS_IDX = tgt_vocab['<eos>']

    with torch.no_grad():
      batch_size, n_tokens = tgt_tokens.shape
      # Get next beams
      src = einops.repeat(src_tokens, 't -> b t', b=tgt_tokens.shape[0])
      predicted = model.forward(src, tgt_tokens)
      predicted = torch.softmax(predicted, dim=-1)
      probs, predicted = predicted[:, -1].topk(k=1, dim=-1)
    
    sentences = []

    for tgt_sentence in tgt_tokens:
      tgt_sentence = list(tgt_sentence)[1:]  # Remove <bos> token
      tgt_sentence = list(takewhile(lambda t: t != EOS_IDX, tgt_sentence))
      tgt_sentence = ' '.join(tgt_vocab.lookup_tokens(tgt_sentence))
      sentences.append(tgt_sentence)

    sentences = [beautify(s) for s in sentences]
    # Join the sentences with their likelihood
    sentences = [(s, p.item()) for s, p in zip(sentences, target_probs)]
    # Sort the sentences by their likelihood
    sentences = [(s, p) for s, p in sorted(sentences, key=lambda k: k[1], reverse=True)]

    return sentences

# Beam search
Beam search is a smarter way of producing a sequence of tokens from
an autoregressive model than just using a greedy search.

The greedy search always choose the most probable token as the unique
and only next target token, and repeat this processus until the *\<eos\>* token is predicted.

Instead, the beam search selects the k-most probable tokens at each step.
From those k tokens, the current sequence is duplicated k times and the k tokens are appended to the k sequences to produce new k sequences.

*You don't have to understand this code, but understanding this code once the TP is over could improve your torch tensors skills.*

---

**More explanations**

Since it is done at each step, the number of sequences grows exponentially (k sequences after the first step, k² sequences after the second...).
In order to keep the number of sequences low, we remove sequences except the top-s most likely sequences.
To do that, we keep track of the likelihood of each sequence.

Formally, we define $s = [s_1, ..., s_{N_s}]$ as the source sequence made of $N_s$ tokens.
We also define $t^i = [t_1, ..., t_i]$ as the target sequence at the beginning of the step $i$.

The output of the model parameterized by $\theta$ is:

$$
T_{i+1} = p(t_{i+1} | s, t^i ; \theta )
$$

Where $T_{i+1}$ is the distribution of the next token $t_{i+1}$.

Then, we define the likelihood of a target sentence $t = [t_1, ..., t_{N_t}]$ as:

$$
L(t) = \prod_{i=1}^{N_t - 1} p(t_{i+1} | s, t_{i}; \theta )
$$

Pseudocode of the beam search:
```
source: [N_s source tokens]  # Shape of [total_source_tokens]
target: [1, <bos> token]  # Shape of [n_sentences, current_target_tokens]
target_prob: [1]  # Shape of [n_sentences]
# We use `n_sentences` as the batch_size dimension

while current_target_tokens <= max_target_length:
    source = repeat(source, n_sentences)  # Shape of [n_sentences, total_source_tokens]
    predicted = model(source, target)[:, -1]  # Predict the next token distributions of all the n_sentences
    tokens_idx, tokens_prob = topk(predicted, k)

    # Append the `n_sentences * k` tokens to the `n_sentences` sentences
    target = repeat(target, k)  # Shape of [n_sentences * k, current_target_tokens]
    target = append_tokens(target, tokens_idx)  # Shape of [n_sentences * k, current_target_tokens + 1]

    # Update the sentences probabilities
    target_prob = repeat(target_prob, k)  # Shape of [n_sentences * k]
    target_prob *= tokens_prob

    if n_sentences * k >= max_sentences:
        target, target_prob = topk_prob(target, target_prob, k=max_sentences)
    else:
        n_sentences *= k

    current_target_tokens += 1
```

In [ ]:
def beautify(sentence: str) -> str:
    """Removes useless spaces.
    """
    punc = {'.', ',', ';'}
    for p in punc:
        sentence = sentence.replace(f' {p}', p)
    
    links = {'-', "'"}
    for l in links:
        sentence = sentence.replace(f'{l} ', l)
        sentence = sentence.replace(f' {l}', l)
    
    return sentence

In [ ]:
def indices_terminated(
        target: torch.FloatTensor,
        eos_token: int
    ) -> tuple:
    """Split the target sentences between the terminated and the non-terminated
    sentence. Return the indices of those two groups.

    Args
    ----
        target: The sentences.
            Shape of [batch_size, n_tokens].
        eos_token: Value of the End-of-Sentence token.

    Output
    ------
        terminated: Indices of the terminated sentences (who's got the eos_token).
            Shape of [n_terminated, ].
        non-terminated: Indices of the unfinished sentences.
            Shape of [batch_size-n_terminated, ].
    """
    terminated = [i for i, t in enumerate(target) if eos_token in t]
    non_terminated = [i for i, t in enumerate(target) if eos_token not in t]
    return torch.LongTensor(terminated), torch.LongTensor(non_terminated)


def append_beams(
        target: torch.FloatTensor,
        beams: torch.FloatTensor
    ) -> torch.FloatTensor:
    """Add the beam tokens to the current sentences.
    Duplicate the sentences so one token is added per beam per batch.

    Args
    ----
        target: Batch of unfinished sentences.
            Shape of [batch_size, n_tokens].
        beams: Batch of beams for each sentences.
            Shape of [batch_size, n_beams].

    Output
    ------
        target: Batch of sentences with one beam per sentence.
            Shape of [batch_size * n_beams, n_tokens+1].
    """
    batch_size, n_beams = beams.shape
    n_tokens = target.shape[1]

    target = einops.repeat(target, 'b t -> b c t', c=n_beams)  # [batch_size, n_beams, n_tokens]
    beams = beams.unsqueeze(dim=2)  # [batch_size, n_beams, 1]

    target = torch.cat((target, beams), dim=2)  # [batch_size, n_beams, n_tokens+1]
    target = target.view(batch_size*n_beams, n_tokens+1)  # [batch_size * n_beams, n_tokens+1]
    return target


def beam_search(
        model: nn.Module,
        source: str,
        src_vocab: Vocab,
        tgt_vocab: Vocab,
        src_tokenizer,
        device: str,
        beam_width: int,
        max_target: int,
        max_sentence_length: int,
    ) -> list:
    """Do a beam search to produce probable translations.

    Args
    ----
        model: The translation model. Assumes it produces linear score (before softmax).
        source: The sentence to translate.
        src_vocab: The source vocabulary.
        tgt_vocab: The target vocabulary.
        device: Device to which we make the inference.
        beam_width: Number of top-k tokens we keep at each stage.
        max_target: Maximum number of target sentences we keep at the end of each stage.
        max_sentence_length: Maximum number of tokens for the translated sentence.

    Output
    ------
        sentences: List of sentences orderer by their likelihood.
    """
    src_tokens = ['<bos>'] + src_tokenizer(source) + ['<eos>']
    src_tokens = src_vocab(src_tokens)

    tgt_tokens = ['<bos>']
    tgt_tokens = tgt_vocab(tgt_tokens)

    # To tensor and add unitary batch dimension
    src_tokens = torch.LongTensor(src_tokens).to(device)
    tgt_tokens = torch.LongTensor(tgt_tokens).unsqueeze(dim=0).to(device)
    target_probs = torch.FloatTensor([1]).to(device)
    model.to(device)

    EOS_IDX = tgt_vocab['<eos>']
    with torch.no_grad():
        while tgt_tokens.shape[1] < max_sentence_length:
            batch_size, n_tokens = tgt_tokens.shape

            # Get next beams
            src = einops.repeat(src_tokens, 't -> b t', b=tgt_tokens.shape[0])
            predicted = model.forward(src, tgt_tokens)
            predicted = torch.softmax(predicted, dim=-1)
            probs, predicted = predicted[:, -1].topk(k=beam_width, dim=-1)

            # Separe between terminated sentences and the others
            idx_terminated, idx_not_terminated = indices_terminated(tgt_tokens, EOS_IDX)
            idx_terminated, idx_not_terminated = idx_terminated.to(device), idx_not_terminated.to(device)

            tgt_terminated = torch.index_select(tgt_tokens, dim=0, index=idx_terminated)
            tgt_probs_terminated = torch.index_select(target_probs, dim=0, index=idx_terminated)

            filter_t = lambda t: torch.index_select(t, dim=0, index=idx_not_terminated)
            tgt_others = filter_t(tgt_tokens)
            tgt_probs_others = filter_t(target_probs)
            predicted = filter_t(predicted)
            probs = filter_t(probs)

            # Add the top tokens to the previous target sentences
            tgt_others = append_beams(tgt_others, predicted)

            # Add padding to terminated target
            padd = torch.zeros((len(tgt_terminated), 1), dtype=torch.long, device=device)
            tgt_terminated = torch.cat(
                (tgt_terminated, padd),
                dim=1
            )

            # Update each target sentence probabilities
            tgt_probs_others = torch.repeat_interleave(tgt_probs_others, beam_width)
            tgt_probs_others *= probs.flatten()
            tgt_probs_terminated *= 0.999  # Penalize short sequences overtime

            # Group up the terminated and the others
            target_probs = torch.cat(
                (tgt_probs_others, tgt_probs_terminated),
                dim=0
            )
            tgt_tokens = torch.cat(
                (tgt_others, tgt_terminated),
                dim=0
            )

            # Keep only the top `max_target` target sentences
            if target_probs.shape[0] <= max_target:
                continue

            target_probs, indices = target_probs.topk(k=max_target, dim=0)
            tgt_tokens = torch.index_select(tgt_tokens, dim=0, index=indices)

    sentences = []
    for tgt_sentence in tgt_tokens:
        tgt_sentence = list(tgt_sentence)[1:]  # Remove <bos> token
        tgt_sentence = list(takewhile(lambda t: t != EOS_IDX, tgt_sentence))
        tgt_sentence = ' '.join(tgt_vocab.lookup_tokens(tgt_sentence))
        sentences.append(tgt_sentence)

    sentences = [beautify(s) for s in sentences]

    # Join the sentences with their likelihood
    sentences = [(s, p.item()) for s, p in zip(sentences, target_probs)]
    # Sort the sentences by their likelihood
    sentences = [(s, p) for s, p in sorted(sentences, key=lambda k: k[1], reverse=True)]

    return sentences

# Training loop
This is a basic training loop code. It takes a big configuration dictionnary to avoid never ending arguments in the functions.
We use [Weights and Biases](https://wandb.ai/) to log the trainings.
It logs every training informations and model performances in the cloud.
You have to create an account to use it. Every accounts are free for individuals or research teams.

In [ ]:
def print_logs(dataset_type: str, logs: dict):
    """Print the logs.

    Args
    ----
        dataset_type: Either "Train", "Eval", "Test" type.
        logs: Containing the metric's name and value.
    """
    desc = [
        f'{name}: {value:.2f}'
        for name, value in logs.items()
    ]
    desc = '\t'.join(desc)
    desc = f'{dataset_type} -\t' + desc
    desc = desc.expandtabs(5)
    print(desc)


def topk_accuracy(
        real_tokens: torch.FloatTensor,
        probs_tokens: torch.FloatTensor,
        k: int,
        tgt_pad_idx: int,
    ) -> torch.FloatTensor:
    """Compute the top-k accuracy.
    We ignore the PAD tokens.

    Args
    ----
        real_tokens: Real tokens of the target sentence.
            Shape of [batch_size * n_tokens].
        probs_tokens: Tokens probability predicted by the model.
            Shape of [batch_size * n_tokens, n_target_vocabulary].
        k: Top-k accuracy threshold.
        src_pad_idx: Source padding index value.
    
    Output
    ------
        acc: Scalar top-k accuracy value.
    """
    total = (real_tokens != tgt_pad_idx).sum()

    _, pred_tokens = probs_tokens.topk(k=k, dim=-1)  # [batch_size * n_tokens, k]
    real_tokens = einops.repeat(real_tokens, 'b -> b k', k=k)  # [batch_size * n_tokens, k]

    good = (pred_tokens == real_tokens) & (real_tokens != tgt_pad_idx)
    acc = good.sum() / total
    return acc


def loss_batch(
        model: nn.Module,
        source: torch.LongTensor,
        target: torch.LongTensor,
        config: dict,
    )-> dict:
    """Compute the metrics associated with this batch.
    The metrics are:
        - loss
        - top-1 accuracy
        - top-5 accuracy
        - top-10 accuracy

    Args
    ----
        model: The model to train.
        source: Batch of source tokens.
            Shape of [batch_size, n_src_tokens].
        target: Batch of target tokens.
            Shape of [batch_size, n_tgt_tokens].
        config: Additional parameters.

    Output
    ------
        metrics: Dictionnary containing evaluated metrics on this batch.
    """
    device = config['device']
    loss_fn = config['loss'].to(device)
    metrics = dict()

    source, target = source.to(device), target.to(device)
    target_in, target_out = target[:, :-1], target[:, 1:]

    # Loss
    pred = model(source, target_in)  # [batch_size, n_tgt_tokens-1, n_vocab]
    pred = pred.view(-1, pred.shape[2])  # [batch_size * (n_tgt_tokens - 1), n_vocab]
    target_out = target_out.flatten()  # [batch_size * (n_tgt_tokens - 1),]
    metrics['loss'] = loss_fn(pred, target_out)

    # Accuracy - we ignore the padding predictions
    for k in [1, 5, 10]:
        metrics[f'top-{k}'] = topk_accuracy(target_out, pred, k, config['tgt_pad_idx'])

    return metrics


def eval_model(model: nn.Module, dataloader: DataLoader, config: dict) -> dict:
    """Evaluate the model on the given dataloader.
    """
    device = config['device']
    logs = defaultdict(list)

    model.to(device)
    model.eval()

    with torch.no_grad():
        for source, target in dataloader:
            metrics = loss_batch(model, source, target, config)
            for name, value in metrics.items():
                logs[name].append(value.cpu().item())

    for name, values in logs.items():
        logs[name] = np.mean(values)
    return logs


def train_model(model: nn.Module, config: dict):
    """Train the model in a teacher forcing manner.
    """
    train_loader, val_loader = config['train_loader'], config['val_loader']
    train_dataset, val_dataset = train_loader.dataset.dataset, val_loader.dataset.dataset
    optimizer = config['optimizer']
    clip = config['clip']
    device = config['device']

    columns = ['epoch']
    for mode in ['train', 'validation']:
        columns += [
            f'{mode} - {colname}'
            for colname in ['source', 'target', 'predicted', 'likelihood']
        ]
    log_table = wandb.Table(columns=columns)


    print(f'Starting training for {config["epochs"]} epochs, using {device}.')
    for e in range(config['epochs']):
        print(f'\nEpoch {e+1}')

        model.to(device)
        model.train()
        logs = defaultdict(list)

        for batch_id, (source, target) in enumerate(train_loader):
            optimizer.zero_grad()

            metrics = loss_batch(model, source, target, config)
            loss = metrics['loss']

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
        
            for name, value in metrics.items():
                logs[name].append(value.cpu().item())  # Don't forget the '.item' to free the cuda memory
            
            if batch_id % config['log_every'] == 0:
                for name, value in logs.items():
                    logs[name] = np.mean(value)

                train_logs = {
                    f'Train - {m}': v
                    for m, v in logs.items()
                }
                wandb.log(train_logs)
                logs = defaultdict(list)
        
        # Logs
        if len(logs) != 0:
            for name, value in logs.items():
                logs[name] = np.mean(value)
            train_logs = {
                f'Train - {m}': v
                for m, v in logs.items()
            }
        else:
            logs = {
                m.split(' - ')[1]: v
                for m, v in train_logs.items()
            }

        print_logs('Train', logs)

        logs = eval_model(model, val_loader, config)
        print_logs('Eval', logs)
        val_logs = {
            f'Validation - {m}': v
            for m, v in logs.items()
        }

        val_source, val_target = val_dataset[ torch.randint(len(val_dataset), (1,)) ]
        val_pred, val_prob = beam_search(
            model,
            val_source,
            config['src_vocab'],
            config['tgt_vocab'],
            config['src_tokenizer'],
            device,  # It can take a lot of VRAM
            beam_width=10,
            max_target=100,
            max_sentence_length=config['max_sequence_length'],
        )[0]
        print(val_source)
        print(val_pred)

        logs = {**train_logs, **val_logs}  # Merge dictionnaries
        wandb.log(logs)  # Upload to the WandB cloud

        # Table logs
        train_source, train_target = train_dataset[ torch.randint(len(train_dataset), (1,)) ]
        train_pred, train_prob = beam_search(
            model,
            train_source,
            config['src_vocab'],
            config['tgt_vocab'],
            config['src_tokenizer'],
            device,  # It can take a lot of VRAM
            beam_width=10,
            max_target=100,
            max_sentence_length=config['max_sequence_length'],
        )[0]

        data = [
            e + 1,
            train_source, train_target, train_pred, train_prob,
            val_source, val_target, val_pred, val_prob,
        ]
        log_table.add_data(*data)
    
    # Log the table at the end of the training
    wandb.log({'Model predictions': log_table})

# Training the models
We can now finally train the models.
Choose the right hyperparameters, play with them and try to find
ones that lead to good models and good training curves.
Try to reach a loss under 1.0.

So you know, it is possible to get descent results with approximately 20 epochs.
With CUDA enabled, one epoch, even on a big model with a big dataset, shouldn't last more than 10 minutes.
A normal epoch is between 1 to 5 minutes.

*This is considering Colab Pro, we should try using free Colab to get better estimations.*

---

To test your implementations, it is easier to try your models
in a CPU instance. Indeed, Colab reduces your GPU instances priority
with the time you recently past using GPU instances. It would be
sad to consume all your GPU time on implementation testing.
Moreover, you should try your models on small datasets and with a small number of parameters.
For exemple, you could set:
```
MAX_SEQ_LEN = 10
MIN_TOK_FREQ = 20
dim_embedding = 40
dim_hidden = 60
n_layers = 1
```

You usually don't want to log anything onto WandB when testing your implementation.
To deactivate WandB without having to change any line of code, you can type `!wandb offline` in a cell.

Once you have rightly implemented the models, you can train bigger models on bigger datasets.
When you do this, do not forget to change the runtime as GPU (and use `!wandb online`)!

In [ ]:
# Checking GPU and logging to wandb

!wandb login

!nvidia-smi

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Wed Mar  8 20:48:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |            

In [ ]:
# Instanciate the datasets

# MAX_SEQ_LEN = 60
# MIN_TOK_FREQ = 2
# train_dataset, val_dataset = build_datasets(
#     MAX_SEQ_LEN,
#     MIN_TOK_FREQ,
#     en_tokenizer,
#     fr_tokenizer,
#     train,
#     valid,
# )


MAX_SEQ_LEN = 60
MIN_TOK_FREQ = 2
train_dataset, val_dataset = build_datasets(
    MAX_SEQ_LEN,
    MIN_TOK_FREQ,
    en_tokenizer,
    fr_tokenizer,
    train,
    valid,
)


print(f'English vocabulary size: {len(train_dataset.en_vocab):,}')
print(f'French vocabulary size: {len(train_dataset.fr_vocab):,}')

print(f'\nTraining examples: {len(train_dataset):,}')
print(f'Validation examples: {len(val_dataset):,}')

English vocabulary size: 11,709
French vocabulary size: 17,645

Training examples: 188,013
Validation examples: 20,891


In [ ]:
# Build the model, the dataloaders, optimizer and the loss function
# Log every hyperparameters and arguments into the config dictionnary

config = {
    # General parameters
    'epochs': 20,
    'batch_size': 256,
    'lr': 1e-3,
    'betas': (0.9, 0.99),
    'clip': 5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',

    # Model parameters
    'n_tokens_src': len(train_dataset.en_vocab),
    'n_tokens_tgt': len(train_dataset.fr_vocab),
    'n_heads': 8,
    # 'dim_embedding': 196,
    # 'dim_hidden': 256,
    # 'n_layers': 3,
    'dim_embedding': 120,
    'dim_hidden': 256,
    'n_layers': 3,
    'dropout': 0.1,
    'model_type': 'GRU',

    # Others
    'max_sequence_length': MAX_SEQ_LEN,
    'min_token_freq': MIN_TOK_FREQ,
    'src_vocab': train_dataset.en_vocab,
    'tgt_vocab': train_dataset.fr_vocab,
    'src_tokenizer': en_tokenizer,
    'tgt_tokenizer': fr_tokenizer,
    'src_pad_idx': train_dataset.en_vocab['<pad>'],
    'tgt_pad_idx': train_dataset.fr_vocab['<pad>'],
    'seed': 0,
    'log_every': 50,  # Number of batches between each wandb logs
}

torch.manual_seed(config['seed'])

config['train_loader'] = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    collate_fn=lambda batch: generate_batch(batch, config['src_pad_idx'], config['tgt_pad_idx'])
)

config['val_loader'] = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    collate_fn=lambda batch: generate_batch(batch, config['src_pad_idx'], config['tgt_pad_idx'])
)



# model = TranslationRNN(
#     config['n_tokens_src'],
#     config['n_tokens_tgt'],
#     config['dim_embedding'],
#     config['dim_hidden'],
#     config['n_layers'],
#     config['dropout'],
#     config['src_pad_idx'],
#     config['tgt_pad_idx'],
#     config['model_type'],
# )


model = TranslationTransformer(
    config['n_tokens_src'],
    config['n_tokens_tgt'],
    config['n_heads'],
    config['dim_embedding'],
    config['dim_hidden'],
    config['n_layers'],
    config['dropout'],
    config['src_pad_idx'],
    config['tgt_pad_idx'],
)


config['optimizer'] = optim.Adam(
    model.parameters(),
    lr=config['lr'],
    betas=config['betas'],
)

weight_classes = torch.ones(config['n_tokens_tgt'], dtype=torch.float)
weight_classes[config['tgt_vocab']['<unk>']] = 0.1  # Lower the importance of that class
config['loss'] = nn.CrossEntropyLoss(
    weight=weight_classes,
    ignore_index=config['tgt_pad_idx'],  # We do not have to learn those
)

summary(
    model,
    input_size=[
        (config['batch_size'], config['max_sequence_length']),
        (config['batch_size'], config['max_sequence_length'])
    ],
    dtypes=[torch.long, torch.long],
    depth=3,
)

Layer (type:depth-idx)                             Output Shape              Param #
TranslationTransformer                             [256, 60, 17645]          --
├─Embedding: 1-1                                   [256, 60, 120]            1,405,080
├─Embedding: 1-2                                   [256, 60, 120]            60,000
├─Dropout: 1-3                                     [256, 60, 120]            --
├─Embedding: 1-4                                   [256, 60, 120]            2,117,400
├─Embedding: 1-5                                   [256, 60, 120]            60,000
├─Dropout: 1-6                                     [256, 60, 120]            --
├─Transformer: 1-7                                 [256, 60, 120]            --
│    └─TransformerEncoder: 2-1                     [256, 60, 120]            --
│    │    └─ModuleList: 3-1                        --                        22,386,600
│    └─TransformerDecoder: 2-2                     [256, 60, 120]            --
│    

In [ ]:
!wandb online  # online / offline to activate or deactivate WandB logging

with wandb.init(
        config=config,
        project='INF8225 - TP3 - 2023',  # Title of your project
        group='Transformers - 3 layer',  # In what group of runs do you want this run to be in?
        save_code=True,
    ):
    train_model(model, config)
# train_model(model, config)

W&B online. Running your script from this directory will now sync to the cloud.


wandb: Currently logged in as: paulovictorcorreia. Use `wandb login --relogin` to force relogin


Starting training for 20 epochs, using cuda.

Epoch 1
Train -   loss: 5.08     top-1: 0.23    top-5: 0.36    top-10: 0.43
Eval -    loss: 5.01     top-1: 0.23    top-5: 0.36    top-10: 0.44
Do you want me to spy on Tom for you?
Tom est.

Epoch 2
Train -   loss: 4.11     top-1: 0.31    top-5: 0.47    top-10: 0.56
Eval -    loss: 3.97     top-1: 0.32    top-5: 0.48    top-10: 0.57
Do you exercise regularly?
J'ai besoin.

Epoch 3
Train -   loss: 3.67     top-1: 0.34    top-5: 0.52    top-10: 0.61
Eval -    loss: 3.55     top-1: 0.35    top-5: 0.53    top-10: 0.62
It looks like somebody was trying to pry open this window.
Je suis désolé.

Epoch 4
Train -   loss: 3.45     top-1: 0.36    top-5: 0.55    top-10: 0.63
Eval -    loss: 3.37     top-1: 0.37    top-5: 0.55    top-10: 0.64
I haven't decided yet if I want to go.
Je suis désolé.

Epoch 5
Train -   loss: 3.34     top-1: 0.37    top-5: 0.56    top-10: 0.64
Eval -    loss: 3.27     top-1: 0.38    top-5: 0.57    top-10: 0.65
The medicine 

Train - loss,█▇▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train - top-1,▁▃▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████████████
Train - top-10,▁▁▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
Train - top-5,▁▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
Validation - loss,█▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Validation - top-1,▁▄▅▆▆▇▇▇▇▇▇█████████
Validation - top-10,▁▄▆▆▇▇▇▇▇▇▇█████████
Validation - top-5,▁▄▆▆▆▇▇▇▇▇▇█████████
Train - loss,2.83781
Train - top-1,0.41124
Train - top-10,0.70782


In [ ]:
sentence = "I liked that we met today!"

preds = beam_search(
    model,
    sentence,
    config['src_vocab'],
    config['tgt_vocab'],
    config['src_tokenizer'],
    config['device'],
    beam_width=10,
    max_target=100,
    max_sentence_length=config['max_sequence_length']
)[:5]

for i, (translation, likelihood) in enumerate(preds):
    print(f'{i}. ({likelihood*100:.5f}%) \t {translation}')

0. (0.00638%) 	 Je n'ai pas le choix.
1. (0.00259%) 	 Je pense qu'il est temps pour nous.
2. (0.00244%) 	 Je n'ai pas besoin d'aide.
3. (0.00235%) 	 Je ne suis pas d'accord avec vous.
4. (0.00226%) 	 Je n'ai pas d'argent.


# Questions
1. Explain the differences between Vanilla RNN, GRU-RNN, and Transformers. 
2. Why is positionnal encoding necessary in Transformers and not in RNNs?
3. Describe the preprocessing process. Detail how the initial dataset is processed before being fed to the translation models.

## Question 1

Vanilla RNN have a single gate activated with hyperbolic tangent activation function. We can the single gate with a weighted sum of the current input $x_t$ and the last hidden state $h_{t-1}$. Therefore, we would have only the parameters $W_{ih}$ associated with the cell input and $W_{hh}$ associated with the previous hidden state. Hence, the equation to compute current gradient is: $h_t = tanh(W_{ih}x_t + W_{hh}h_{t-1} + b)$. This configuration makes harder for the gradients to propagate over time, either making the gradient vanish or explode given a large enough sequence. To overcome this, we can perform gradient cliping, which is already done in this notebook. However, this makes the training much slower and worse, thus we could make an architecture change by using Gated Recurrent Unit (GRU) RNN.

GRU RNN have more gates in their cells. This architecture change makes the gradient vanishing or exploding more difficult, thus it can look further in time than Vanilla RNN. The GRU has two gates: update ($z_t$) and reset ($r_t$) gates. They are responsible to determine what information will be forgotten and what information will be added to the memory of the network. Then we perform a linear weighted operation ($n_t$) with the reset gate and the input gate and the compute the actual $h_t$ of the current timestep. GRU have the following equations which $\sigma$ represents the sigmoid activation:

$$r_t = σ(W_{ir}x_t + W_{hr}h_{t-1} + b)$$
$$z_t = σ(W_{iz}x_t + W_{hz}h_{t-1} + b)$$
$$n_t = tanh(W_{in}x_t + r_t*(W_{hn}h_{t-1}) + b)$$
$$h_t = (1-z_t)*n_t + z_t*h_{t-1}$$

However, for long enough sequences, there would still be gradient issues even with the correct precautions. The RNNs in general have a lot of difficult to associate sequence elements in different parts that would build a good enough context to interpret the sequence. One way around would be to look at the whole sequence at once and build a context that a model which words refers to others without the risk of gradient vanishing. That's what transformers adresses.

Transformers peform a special operation called attention, which basically compares each element to one another at once. Thus, this operation provides a powerful context invariant to time and avoids the learning problems we experience with RNNs. We then feed the decoded context of the transformer to a multilayer perceptron network and perform the desired task.






## Question 2

Positional encoders aren't necessary in RNNs because we feed the RNNs sequentially at each time step. Therefore, the model knows the order of the words prior to the translation because it is encoded in the order we feed the network. However, transformers require positional encoding for exactly the reason it makes the architecture stronger to training issues. Since we feed the transformers the whole sequence at once, the model can't know the order of the words, hence we'd need a way to make the model aware of the model. And that's exactly what the positional encoding does: it adds to the current word embedding an information about the position which it is found in the sequence. Without this the model has a difficult time on translating the sentences.

## Question 3

This project of translation from english to french consists of a NLP task. Therefore, the preprocessing steps are:

* Filter sentences to remove '\\n' tokens from the sentences to have tidy words for the tokenization step. 

* Tokenization. This process consists of mapping the words into numbers up until the vocabulary size of the dataset. For example, we could map the words \["Hello", "World", "Fish"\] into de dictionary  \{"Hello":0, "World":1, "Fish":2\}. Each of these integer indexes will feed an embedding layer that will feed the neural network models. The notebook performs the spacy tokenization, an NLP framework that already knows the tokens of some languages, for both english and french.

* Add the special tokens \['\<unk>', '\<pad>', '\<bos>', '\<eos>'\] to the sentences to fill the tokenized vectors so that all sentences will have the same length. The '\<unk>' token stands for words outside the vocabulary, while the '\<pad>' token simply fills the tokenized sentences after '\<eos>' until they have the max sentence length. Finally,  '\<bos>' and '\<eos>' marks the begining and end of the sentences respectively.



# Small report - experiments
Once everything is working fine, you can explore aspects of these models and do some research of your own into how they behave.

For exemple, you can experiment with the hyperparameters.
What are the effect of the differents hyperparameters with the final model performance? What about training time?

What are some other metrics you could have for machine translation? Can you compute them and add them to your WandB report?

Those are only examples, you can do whatever you think will be interesting.
This part account for many points, *feel free to go wild!*

---
# Introduction

This report displays the experimental results obtained for the translation model with Vanilla RNN, GRU RNN and transformers. We will display the results for each type of model according to the size of the model. Each size will depend on the number of layers, the embedding size and the hidden size of each model. For example, a small model would have 1 or 2 stacked layers, with an embedding size of 40 and a hidden size of 60. However, we have a constraint of model size given by the virtual machine used to train. Therefore, we will use models with a maximum number of layers of 3.

Also, we will present small modifications on implementation to the original transformer in order to simplify the model.

## Positional Encoding Modification

Transformers require positional encoding to inject information about relative and absolute position of the words in a sentence. They are necessary because the transformers architecture have no clue of the order of the words in the sequence fed to the model. Therefore, the authors of "Attention is All You Need" propose a positional encoding to make the transformer aware of the position of the words. Hence, they do this by encoding words on even and odd positions of the sentence with the following equations:

$$PE_{(pos, 2i)} = \sin(pos/10000^{2i/d_{model}})$$

$$PE_{(pos, 2i+1)} = \cos(pos/10000^{2i/d_{model}})$$

which $d_{model}$ stands for the embedding dimension, $pos$ is the position and $i$ is the dimension. We use the first equation on words in even positions and the second on words in odd positions.

However, we made a small modification based on Alladin Person's [implementation](https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/more_advanced/transformer_from_scratch/transformer_from_scratch.py), in which he creates a vector from zero to the sequence length and expand it to have the shape (batch_size, sequence_length). After that, we feed this vector to a positional embedding layer and sum to the actual sequence embedding layer.

We will see that this implementation worked as the results will confirm.

## Experimental Configuration

We will perform a parameter grid search with the following distribution of parameters 

The dataset generation:
* Max Sequence Size: 60
* Min Token Frequency: 2

Training Parameters:
* Adam Optimizer
* Learning Rate: 0.001
* Batch size: 256
* Epochs: 20

Vanilla-RNN and GRU-RNN configuration:
* Number of layers: \[1, 2, 3\]
* Hidden size: 256
* Embedding size: 120

Transformer's configuration:
* Number of layers: \[1, 2, 3\]
* Hidden size: 256
* Embedding size: 120
* Number of attention heads: 8

## Results And Discussion

We had to buy Google Colab Pro to perform more refined experiments, especially when there were more layers. Even though, we couldn't perform more refined experiments by changing hidden size and embedding layer since there were limited resources.

Having said that, the experiments were performed in Google Colab Pro GPUs, either NVIDIA T4 Tensor core or V100 and A100 premium GPUs. The hyperparameters we searched for were only the number of layers in the recurrent networks, and the number of stacked encoders and decoders for the transformer.

We will present the results for each network architecture as well as the hyperparameters used in each one of them.

### Vanilla RNN


Train results: 


dim_embedding|dim_hidden | num. layers | Train - top-1  |Train - top-10| Train - top-5|
-------------------|------------------|------------------| ------------------|------------------|------------------|
120|256   | 1 | 0.46 | 0.67 | 0.75 |
120|256  | 2 | 0.42 | 0.64 | 0.73 |
120|256  | 3 | 0.43 | 0.64 | 0.73 | 


Test results:


dim_embedding|dim_hidden | num. layers | Test - top-1  |Test - top-10| Test - top-5|
-------------------|------------------|------------------| ------------------|------------------|------------------|
120|256   | 1 | 0.45 | 0.66 | 0.74 |
120|256  | 2 | 0.42 | 0.63 | 0.72 |
120|256  | 3 | 0.44 | 0.65 | 0.73 | 


### GRU RNN
Train results: 


dim_embedding|dim_hidden | num. layers | Train - top-1  |Train - top-10| Train - top-5|
-------------------|------------------|------------------| ------------------|------------------|------------------|
120|256   | 1 | 0.55 | 0.76 | 0.82 |
120|256  | 2 | 0.60 | 0.82 | 0.87 |
120|256  | 3 | 0.63 | 0.84 | 0.89 | 


Test results:


dim_embedding|dim_hidden | num. layers | Test - top-1  |Test - top-10| Test - top-5|
-------------------|------------------|------------------| ------------------|------------------|------------------|
120|256   | 1 | 0.56 | 0.76 | 0.82 |
120|256  | 2 | 0.62 | 0.82 | 0.87 |
120|256  | 3 | 0.65 | 0.85 | 0.89 | 

Comparing the results with the Vanilla RNNs, GRUs are more powerful for the translation task than Vanilla RNNs. We can assume that one of the reasons is the GRU's architecture, which allows the network to have more paramaters and make stronger associations in time. 

However, since we could train for only 20 epochs, we could not see how powerful the network could've become. But given enough resources to train, GRUs can output really robust models.

### Transformers

Train results: 


dim_embedding|dim_hidden | num. layers | Train - top-1  |Train - top-10| Train - top-5|
-------------------|------------------|------------------| ------------------|------------------|------------------|
120|256   | 1 | 0.74 | 0.91 | 0.94 |
120|256  | 2 | 0.71 | 0.88 | 0.92 |
120|256  | 3 | 0.41 | 0.62 | 0.71 | 


Test results:


dim_embedding|dim_hidden | num. layers | Test - top-1  |Test - top-10| Test - top-5|
-------------------|------------------|------------------| ------------------|------------------|------------------|
120|256   | 1 | 0.76 | 0.91 | 0.93 |
120|256  | 2 | 0.73 | 0.89 | 0.92 |
120|256  | 3 | 0.42 | 0.62 | 0.71 | 


Transformers play a completly different game on the translation task. For just a single encoder-decoder translator, it outperformed both RNN architectures by a comfort amount. It also had a good margin for improvement given enough resources.

However, when we increased the number of stacked encoders and decoders, the performance fell significantly. A possible reason could be the increased amount of parameters when we stack layers. In our implementation, they made convergence slow and given enough epochs it could outperform using a single layer. However, we had limited resources, therefore we assume that for this problem 1 layer is better exaclty because it converges faster and require less resources to complete.

# Conclusion

After thoroughly performing experiments, we conclude that Transformers outperform recurrent neural networks in translation. While the RNN's sequential cascade processing introduces the risk of gradient vanishing or exploding, or even lose context of the sentence, transformer's architecture allows to compute the whole sequence at once without this risk. Even by the necessity of positional embedding encoding to make the model aware of the words position in a sentence, transformers produced a more robust model with a faster convergence over the other models.

Other experiments that we could have performed include a more refined hyperparameter search with more epochs to assess the model's performance, but it requires a lot of computational resources that are unavailable.


